# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
city_df = pd.read_csv("city_data.csv")
city_df.head()


,City,lat,lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Mariestad,58.7097,13.8237,4.27,82,17,8.44,SE,1644058555
1,Ubari,26.5921,12.7805,17.59,17,0,5.19,LY,1644058635
2,Ancud,-41.8697,-73.8203,11.79,87,3,0.98,CL,1644058572
3,Takoradze,4.8845,-1.7554,29.94,64,11,2.22,GH,1644058720
4,Bubaque,11.2833,-15.8333,25.25,43,0,2.39,GW,1644058791


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [68]:
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


In [83]:
locations = city_df[["lat", "lng"]]
weight = city_df["Humidity"].astype(float)
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=weight,
                                 max_intensity= city_df["Humidity"].max())
                                 


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
#city_df["Max_Temp"] = city_df["Max_Temp"].astype('float64')
ideal= city_df[city_df["Max_Temp"]>25]
ideal= ideal[ideal["Wind_Speed"]<10]
ideal= ideal[ideal["Cloudiness"]== 0]
len(ideal)


9

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [70]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'keyword': "5000 meters",
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}

In [50]:
hotelname =[]
hoteladdy = []

for index, row in ideal.iterrows():

    # get restaurant type from df
    location = f'{row["lat"]}, {row["lng"]}'
    print(location)

    # add keyword to params dict
    params['location'] = location
    city = row["City"]

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city} {results[0]['name']}.")
        
        hotelname.append(results[0]['name'])
        hoteladdy.append(results[0]['vicinity'])
        
    except (KeyError, IndexError):
        hotelname.append("NA")
        hoteladdy.append("NA")
        print("Missing field/result... skipping.")
        
    print("------------")

ideal["Hotel Name"] = hotelname
ideal["Hotel Address"] = hoteladdy

11.2833, -15.8333
Retrieving Results for Index 4: Bubaque.
Closest hotel in Bubaque Saldomar B&Biosphere.
------------
-33.9258, 18.4232
Retrieving Results for Index 26: Cape Town.
Closest hotel in Cape Town Esme Witbooi Coaching.
------------
-25.2406, 130.9889
Retrieving Results for Index 238: Yulara.
Missing field/result... skipping.
------------
-28.7667, 114.6
Retrieving Results for Index 277: Geraldton.
Closest hotel in Geraldton Houtman Abrolhos Conservation Network.
------------
-33.65, 115.3333
Retrieving Results for Index 403: Busselton.
Closest hotel in Busselton JAB AGENCY.
------------
25.5167, 69.7667
Retrieving Results for Index 462: Chhor.
Closest hotel in Chhor Ali Nawaz Nohri Autoz Chhor.
------------
17.0151, 54.0924
Retrieving Results for Index 488: Salalah.
Closest hotel in Salalah Nepali Restaurant.
------------
14.9515, -16.8129
Retrieving Results for Index 497: Tivaouane.
Closest hotel in Tivaouane Palais des hôtes de Tivaouane.
------------
11.75, -13.2
Retriev

In [72]:
hotel_df = ideal[["City","lat","lng","Hotel Name", "Hotel Address","Country"]]
hotel_df.rename(columns = {"lat":"Lat", "lng": "Lng"}, inplace = True)

hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df.to_csv("hotel_df.csv", index=False)
hotel_df

,City,Lat,Lng,Hotel Name,Hotel Address,Country
4,Bubaque,11.2833,-15.8333,Saldomar B&Biosphere,"Puerto Dr Bubaque, Bubaque",GW
26,Cape Town,-33.9258,18.4232,Esme Witbooi Coaching,"Darling Street, Cape Town",ZA
277,Geraldton,-28.7667,114.6000,Houtman Abrolhos Conservation Network,"Gregory Street, Beachlands",AU
403,Busselton,-33.6500,115.3333,JAB AGENCY,"Thomas Street, West Busselton",AU
462,Chhor,25.5167,69.7667,Ali Nawaz Nohri Autoz Chhor,"GQ88+QM5, Chor",PK
488,Salalah,17.0151,54.0924,Nepali Restaurant,"238R+3W8, Salalah",OM
497,Tivaouane,14.9515,-16.8129,Palais des hôtes de Tivaouane,"X52P+QR9, Tivaouane",SN
511,Gaoual,11.7500,-13.2000,Mosquée de Gaoual,"QQ2X+FJV, Gaoual",GN


In [85]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [86]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))